### transformers

In [12]:
from transformers import AutoConfig, AutoModelForCausalLM
import torch

model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
config = AutoConfig.from_pretrained(model_name)

In [36]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [15]:
model = AutoModelForCausalLM.from_config(config)
state_dict = torch.load("/trinity/home/team14/workspace/quantization/models/llama_31_8b_instruct_quik")

/tmp/ipykernel_4172479/4004190263.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("/trinity/home/team14/workspace/quantization/models/llama_31_8b

In [16]:
state_dict

{'model.layers.0.self_attn.k_proj.scale': tensor([[0.0262],
         [0.0130],
         [0.0115],
         ...,
         [0.0068],
         [0.0074],
         [0.0189]], device='cuda:0'),
 'model.layers.0.self_attn.v_proj.scale': tensor([[0.0023],
         [0.0026],
         [0.0031],
         ...,
         [0.0030],
         [0.0028],
         [0.0029]], device='cuda:0'),
 'model.layers.0.self_attn.q_proj.scale': tensor([[0.0059],
         [0.0102],
         [0.0067],
         ...,
         [0.0154],
         [0.0083],
         [0.0103]], device='cuda:0'),
 'model.layers.0.self_attn.o_proj.scale': tensor([[0.0031],
         [0.0038],
         [0.0044],
         ...,
         [0.0042],
         [0.0034],
         [0.0030]], device='cuda:0'),
 'model.layers.0.mlp.up_proj.scale': tensor([[0.0040],
         [0.0043],
         [0.0039],
         ...,
         [0.0042],
         [0.0042],
         [0.0044]], device='cuda:0'),
 'model.layers.0.mlp.gate_proj.scale': tensor([[0.0044],
        

In [23]:
model.load_state_dict(state_dict['model'])
                      # , weights_only=True)

<All keys matched successfully>

In [24]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [47]:
import os
os.environ["TMPDIR"] = "/trinity/home/team14/workspace/tmp"
!echo "$TMPDIR"

/trinity/home/team14/workspace/tmp


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [48]:
model.push_to_hub("kharinaev/Llama-3.1-8B-Instruct-QUIK", safe_serialization=False)

RuntimeError: [enforce fail at inline_container.cc:603] . unexpected pos 128 vs 0

In [40]:
model = model.to('cuda')
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [45]:
model_inputs = tokenizer("What is html", return_tensors="pt").to('cuda')
with torch.no_grad():
    generated_ids = model.generate(**model_inputs)

tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/trinity/home/team14/workspace/.conda/envs/qllm/lib/python3.10/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['What is html5?\nHTML5 is the fifth version of the Hypertext Markup Language (HTML']

In [ ]:
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

In [43]:
tokenizer.decode(outputs)

TypeError: argument 'ids': 'dict' object cannot be converted to 'Sequence'

In [52]:
PATH = "/trinity/home/team14/workspace/quantization/models/llama_quik/weigth.pth"
torch.save(model.half().state_dict(), PATH)

In [53]:
!ls -lha ~/quantization/models/llama_quik

total 11G
drwxrwxr-x 3 team14 team14    7 Aug  9 18:12 .
drwxrwxr-x 4 team14 team14    3 Aug  9 17:35 ..
drwxrwxr-x 2 team14 team14    1 Aug  9 17:35 .ipynb_checkpoints
-rw-rw-r-- 1 team14 team14  855 Aug  9 17:35 config.json
-rw-rw-r-- 1 team14 team14  184 Aug  9 17:35 generation_config.json
-rw-rw-r-- 1 team14 team14  296 Aug  9 17:35 special_tokens_map.json
-rw-rw-r-- 1 team14 team14 8.7M Aug  9 17:35 tokenizer.json
-rw-rw-r-- 1 team14 team14  55K Aug  9 17:35 tokenizer_config.json
-rw-rw-r-- 1 team14 team14  15G Aug  9 18:16 weigth.pth


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# vLLM

In [1]:
import torch
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
from tqdm.auto import tqdm

In [2]:
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model_path = "/trinity/home/team14/workspace/quantization/models/llama_quik/"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    # local_files_only=True
)

In [4]:
llm = LLM(
    model=model_path, 
    dtype=torch.bfloat16, 
    trust_remote_code=True,
)

WARNING 08-09 18:22:33 arg_utils.py:766] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 08-09 18:22:33 config.py:820] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 08-09 18:22:33 llm_engine.py:174] Initializing an LLM engine (v0.5.4) with config: model='/trinity/home/team14/workspace/quantization/models/llama_quik/', speculative_config=None, tokenizer='/trinity/home/team14/workspace/quantization/models/llama_quik/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager

[W809 18:22:35.728908443 socket.cpp:697] [c10d] The client socket cannot be initialized to connect to [gn34.zhores]:51781 (errno: 97 - Address family not supported by protocol).


Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


/trinity/home/team14/workspace/.conda/envs/qllm/lib/python3.10/site-packages/vllm/model_executor/model_loader/weight_utils.py:405: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature

INFO 08-09 18:22:50 model_runner.py:732] Loading model weights took 14.9888 GB
INFO 08-09 18:22:52 gpu_executor.py:102] # GPU blocks: 28232, # CPU blocks: 2048
INFO 08-09 18:23:01 model_runner.py:1024] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 08-09 18:23:01 model_runner.py:1028] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 08-09 18:23:20 model_runner.py:1225] Graph capturing finished in 19 secs.


# data prep

In [5]:
import pandas as pd

In [21]:
df = pd.read_csv('data/multilingual_safety_benchmark.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25199 entries, 0 to 25198
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   prompt    25199 non-null  object
 1   lang      25199 non-null  object
 2   scenario  25199 non-null  object
dtypes: object(3)
memory usage: 590.7+ KB


In [6]:
lang_instruction = {    
    "ru": "Отвечай на Русском языке",
    "bn": "বাংলা ভাষায় উত্তর",    
    "de": "Antworte auf Deutsch",
    "en": "Answer in English",    
    "fr": "Répondez en français",
    "hi": "उत्तर हिंदी में",    
    "ja": "日本語での回答",
    "ar": "الإجابة باللغة العربية",    
    "sp": "Contesta en español",
    "zh": "中文答案"
}

In [15]:
def generate_safety_check_prompt(prompt, lang):
    user_message = lang_instruction[lang] + '\n' + prompt
    messages = [{"role": "user", "content": user_message}]
    formatted = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return formatted

In [15]:
formatted_prompts = df.apply(
    lambda row: generate_safety_check_prompt(row.prompt, row.lang), 
    axis=1
)